<a href="https://colab.research.google.com/github/RobAltena/AdventOfCode2023/blob/main/Day3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import re
import numpy as np

data = requests.get('https://raw.githubusercontent.com/RobAltena/AdventOfCode2023/main/advent_day3_input.txt').text

lines = data.split('\n')[:-1]
max_line = len(lines)-1

#Part 1
answer = 0
for lineno, line in enumerate(lines):
    numbers = [(m.start(),  m.group() )for m in re.finditer(r"\d+", line)]

    line_min =  lineno -1  if  (lineno > 0)  else 0
    line_max =  lineno + 1  if  (lineno < max_line)  else max_line

    l = len(line) - 1
    for (pos, number) in numbers:
        pos_min = pos - 1 if (pos > 0) else 0
        pos_max = pos + len(number) + 1  if (pos + len(number) <= l ) else pos + len(number)

        test_string = ""
        for test_line in lines[line_min:line_max +1]:
            test_string =  test_string + test_line[pos_min:pos_max]


        if not (re.match('^[\.\d]+$', test_string)) :
            answer += int(number)

print(answer)

In [ ]:
#Part 2
max_len = len(lines[0]) +1
potential_gears = [ m.start() for m in re.finditer(r"\*", data)]
potential_gears = list(map(lambda x: (int(x/max_len), x % max_len), potential_gears))

# Check to make sure there are no gears on the edges, which makes it easier.
for x, y in potential_gears:
    if x in {0, max_len - 1} or y in {0, max_len - 1}:
        raise Exception(f"There is a gear on the edge. ({x} - {y})")

# Convert multiline string into a 2D NumPy array of individual characters, excluding the last empty line
points_arr = np.array((list(map(list, data.split('\n')[:-1]))), dtype=str)

# Check to make sure all  gears are actually gears in the numpy array.
for x,y in potential_gears:
    if (points_arr[x][y] !='*'):
        raise Exception(f"One of the gears is not a *. ({x} - {y})")

# Not the prettiest function, but it works. get the number form x,y by expanding left and right.
def parsenumber(x,y):
    if points_arr[x][y] not in '0123456789':
        raise Exception ("Called paresenumber with invalid character. (" +str(x) + " - " + str(y) + ") " +  points_arr[x][y])
    res = [points_arr[x][y]]
    y2 = y +1
    while (y2 <= max_line):
        char = points_arr[x][y2]
        if (char in '.*') : break
        res.append(char)
        y2 += 1

    y2 = y - 1
    while (y2 >= 0):
        char = points_arr[x][y2]
        if (char in '.*') : break
        res.insert(0, char)
        y2 -= 1

    return int(''.join(res))

# Another not so pretty function. Calculate the actual gear ratio for a gear. 0 if not a gear.
def calc_gear_ratio(g):
    (x,y) = g
    if (points_arr[x,y] != "*"):
        print("gear at (" + str(x) + ', ' + str(y) + ') is not a *')
        return 0

    arr = points_arr[x-1:x+2,y-1:y+2]
    arr_digits = np.char.isdigit(arr)

    count = 1 if arr_digits[0][1] else np.count_nonzero(arr_digits[0])
    count += np.count_nonzero(arr_digits[1])
    count += 1 if arr_digits[2][1] else np.count_nonzero(arr_digits[2])
    if (count != 2):
        # gear  is not a gear. it must have 2 neighbouring digits.
        return 0

    # Now we know this is a real gear. Now lets find the 2 numbers.
    nodes = []
    if arr_digits[0][1] :
        nodes.append((-1, 0))
    else:
        if arr_digits[0][0] :  nodes.append((-1,-1))
        if arr_digits[0][2] :  nodes.append((-1,1))

    if arr_digits[1][0] :  nodes.append((0, -1))
    if arr_digits[1][2] :  nodes.append((0, 1))

    if arr_digits[2][1] :
            nodes.append((1,0))
    else:
            if arr_digits[2][0] :  nodes.append((1,-1))
            if arr_digits[2][2] :  nodes.append((1,1))

    gear_ratio = 1
    for node in nodes:
         x1 = x + node[0]
         y1 = y + node[1]
         gear_ratio = gear_ratio * parsenumber(x1,y1)

    return gear_ratio

# finally print the answer.
sum(map(calc_gear_ratio, potential_gears))